In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
path = '/content/gdrive/My Drive/indeed scraping'
os.chdir(path)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def extract(page):
    url = f'https://ca.indeed.com/jobs?q=Data+Analyst&l=Toronto%2C+ON&sort=date&start={page}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

In [ ]:
def transform(soup):
    divs = soup.find_all('div', class_ = 'job_seen_beacon')
    for item in divs:
        title = item.find('h2').text.strip()[3:] # BE CAREFUL with this! Change if not sure all the rows cotain 'new'!
        company = item.find('span', class_ = 'companyName').text.strip()
        location = item.find('div', class_= 'companyLocation').text.strip()
        try:
            salary = item.find('div', class_ = 'metadata salary-snippet-container').text.strip()
        except:
            salary = ''
        summary = item.find('div', class_ = 'job-snippet').text.strip().replace('\n', ' ')
        
        job = {
            'title': title,
            'company': company,
            'location': location,
            'salary': salary,
            'summary': summary
        }
        joblist.append(job)
    return

In [ ]:
def get_multi_pages(pages):
    for i in range(0, pages, 10):
        c = extract(i)
        transform(c)

In [ ]:
joblist = []
get_multi_pages(100)

In [ ]:
job = pd.DataFrame(joblist)

In [ ]:
job.head()

,title,company,location,salary,summary
0,Junior Project Manager / Senior Project Analys...,CorGTA Inc.,"Toronto, ON•Temporarily Remote",$42 an hour,We are looking for a Junior Project Manager / ...
1,"Data Analyst, Growth",NorthOne,"Toronto, ON•Remote",,Our company's purpose is to eliminate the pain...
2,Big Data Analyst,Bell Canada,"Toronto, ON",,Ability to work across data functions of data ...
3,Senior Data Control Analyst,Lactalis,"Toronto, ON•Remote",,"Ability to create mass data changes, data anal..."
4,Business Analyst,Intact,"Toronto, ON",,An industry leading Employee Share Purchase Pl...


In [ ]:
job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     150 non-null    object
 1   company   150 non-null    object
 2   location  150 non-null    object
 3   salary    150 non-null    object
 4   summary   150 non-null    object
dtypes: object(5)
memory usage: 6.0+ KB


In [ ]:
job.to_csv('job.csv', )